# Group Project, project_5, Finding Danger Spots in the US amidst COVID-19
Matt Paterson, hello@hireMattPaterson.com

## Looking at data around natural disasters, what risk does each geographic area have?

**In this project we analyze COVID-19 data, data around natural disasters, and other factors to identify hot spots that may or may not be at a higher risk than other geographic regions in 2020 and beyond.  We will create an overlay map that allows a user to drill down by location and/or by type of threat (ie Earthquake, Hurricane, Tornado, COVID-19 outbreak, etc) to see what the relative risk of each area may be.**

***THIS NOTEBOOK is an early step in to the full project, and will specifically look at a dataset that was too lean to be used in our full group project.  I'll walk through the steps needed here to create a linear regression model.***

## Data 

This notebook is created to pull data specifically relating to the COVID-19 pandemic.  I will pull in a dataset and run a preliminary EDA and model on it.  

For this particular notebook, it was not necessary to create a web scraper or an API-interactive function as the data is already organized in a git repository thanks to Johns Hopkins University. However as you'll see below this dataset is not robust enough for our purposes.

In [1]:
import pandas as pd                 # import the pandas library
import numpy as np                  # import the numpy library
import matplotlib.pyplot as plt     # import the matplotlib library
import seaborn as sns               # import the seaborn library

In [2]:
# ENTER YESTERDAY'S DATE as a string below IN THE FORM MM-DD-YYYY
yesterday = '07-31-2020'
today = '2020-08-01'

path_to_home = '../'
path_to_subsets = path_to_home + 'data/BIG-QUERY/'
report = path_to_subsets + today + '-county.csv'

## Read in a csv of the dataset

In [3]:
df = pd.read_csv(report)

In [4]:
# Look at the first five rows of data to get a general idea of what the dataset looks like
df.head()

,province_state,country_region,date,latitude,longitude,sub_region1_name,location_geom,confirmed,deaths,recovered,active,fips,admin_2,combined_key
0,Wisconsin,United States of America,2020-08-01,44.90,-89.76000,Wisconsin,POINT(-89.76 44.9),NaN,NaN,NaN,NaN,55073,Marathon County,US_WI_55073
1,Wisconsin,United States of America,2020-08-01,45.34,-88.00000,Wisconsin,POINT(-88 45.34),NaN,NaN,NaN,NaN,55075,Marinette County,US_WI_55075
2,Wisconsin,United States of America,2020-08-01,43.82,-89.39000,Wisconsin,POINT(-89.39 43.82),NaN,NaN,NaN,NaN,55077,Marquette County,US_WI_55077
3,Wisconsin,United States of America,2020-08-01,45.02,-88.70000,Wisconsin,POINT(-88.7 45.02),NaN,NaN,NaN,NaN,55078,Menominee County,US_WI_55078
4,Wisconsin,United States of America,2020-08-01,43.00,-87.96713,Wisconsin,POINT(-87.96713 43),NaN,NaN,NaN,NaN,55079,Milwaukee County,US_WI_55079


In [5]:
# look at the pandas.DataFrame.shape attribute to see the (rows, columns) that make up our dataset
df.shape

(3220, 14)

## Exploratory Data Analysis
Fifty-eight rows is not a lot of data. This dataset tells the story of the US States and territories. This level of data will not satisfy the requirements of our data problem. We'll need to drill down to the local level--the county level at the least, zip code or even neighborhood at the best--in order to learn what we need to learn to make our interactive heatmaps relevant.

For this notebook I'll continue the EDA, visualizations, and investigations in hopes that this same methodology will scale to the correct dataset.

1. Identify any null values within the dataset 
2. Clear or convert any columns that are non-numeric 
3. Graph or plot as much of the data as is needed to understand it

In [6]:
# Show all null values
df.isna().sum()

province_state         0
country_region         0
date                   0
latitude               0
longitude              0
sub_region1_name       0
location_geom          0
confirmed           3220
deaths              3220
recovered           3220
active              3220
fips                   0
admin_2                0
combined_key           0
dtype: int64

We see that we have null values in the following rows:
<ul>
    <li>Confirmed</li>
    <li>Deaths</li>
    <li>Recovered</li>
    <li>Active</li>
    
</ul>

We'll have to import the cases and deaths data from another table and marry them here using pd.concat since those are pretty important pieces of the data

In [7]:
# List out the datatypes
df.dtypes

province_state       object
country_region       object
date                 object
latitude            float64
longitude           float64
sub_region1_name     object
location_geom        object
confirmed           float64
deaths              float64
recovered           float64
active              float64
fips                  int64
admin_2              object
combined_key         object
dtype: object

Many of our columns are non-numeric data, but this is OK as we'll combine some of them to form the new index and drop some as well once we have married the other data table to this one.

## Bring in the second dataset


In [8]:
# ENTER YESTERDAY'S DATE as a string below IN THE FORM MM-DD-YYYY
yesterday = '2020-07-31'
today = '2020-08-01'

path_to_home = '../'
path_to_subsets = path_to_home + 'data/BIG-QUERY/'
report_2 = path_to_subsets + yesterday + '-comprehensive.csv'

In [9]:
df_2 = pd.read_csv(report_2)

In [10]:
df_2.head()

,date,location_key,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,...,datacommons_id,openstreetmap_id,latitude,longitude,location_geometry,average_temperature_celsius,minimum_temperature_celsius,maximum_temperature_celsius,rainfall_mm,snowfall_mm
0,2020-07-31,UM,UM,United States Minor Outlying Islands,NaN,NaN,NaN,NaN,UM,UMI,...,country/UMI,2185386.0,19.300000,166.633333,POINT(166.633333 19.3),NaN,NaN,NaN,NaN,NaN
1,2020-07-31,VI,VI,United States Virgin Islands,NaN,NaN,NaN,NaN,VI,VIR,...,country/VIR,286898.0,18.333333,-64.833333,POINT(-64.833333 18.333333),NaN,NaN,NaN,NaN,NaN
2,2020-07-31,US_VA,US,United States of America,VA,Virginia,NaN,NaN,US,USA,...,geoId/51,224042.0,37.500000,-79.000000,POINT(-79 37.5),22.000000,20.722222,23.0,0.0,NaN
3,2020-07-31,US_VA_51035,US,United States of America,VA,Virginia,51035.0,Carroll County,US,USA,...,geoId/51035,2532620.0,36.730000,-80.730000,POINT(-80.73 36.73),NaN,NaN,NaN,NaN,NaN
4,2020-07-31,US_WI,US,United States of America,WI,Wisconsin,NaN,NaN,US,USA,...,geoId/55,165466.0,44.500000,-89.500000,POINT(-89.5 44.5),22.277778,18.611111,25.5,0.0,NaN


In [11]:
df_2.shape

(3282, 45)

In [12]:
counties = df_2[df_2.subregion2_name.notna()]
counties.shape

(3220, 45)

In [13]:
counties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3220 entries, 3 to 3281
Data columns (total 45 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   date                                3220 non-null   object 
 1   location_key                        3220 non-null   object 
 2   country_code                        3220 non-null   object 
 3   country_name                        3220 non-null   object 
 4   subregion1_code                     3220 non-null   object 
 5   subregion1_name                     3220 non-null   object 
 6   subregion2_code                     3220 non-null   float64
 7   subregion2_name                     3220 non-null   object 
 8   iso_3166_1_alpha_2                  3220 non-null   object 
 9   iso_3166_1_alpha_3                  3220 non-null   object 
 10  aggregation_level                   3220 non-null   int64  
 11  new_confirmed                       3187 no

## FIPS data:  The names and code numbers of the county

In this dataset, df_2, the FIPS code is denoted as 'subregion2_code'

In [14]:
# subregion2_code  ,subregion2_name
df_2[['subregion2_code', 'subregion2_name']]
    

,subregion2_code,subregion2_name
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,51035.0,Carroll County
4,NaN,NaN
...,...,...
3277,13153.0,Houston County
3278,13165.0,Jenkins County
3279,18003.0,Allen County
3280,18037.0,Dubois County


In [15]:
df_2[['latitude', 'longitude']]

,latitude,longitude
0,19.300000,166.633333
1,18.333333,-64.833333
2,37.500000,-79.000000
3,36.730000,-80.730000
4,44.500000,-89.500000
...,...,...
3277,32.460000,-83.670000
3278,32.790000,-81.960000
3279,41.090000,-85.060000
3280,38.360000,-86.880000


## Export this df_2 to a csv for use in a more useful notebook:



In [16]:
df_2.to_csv('../data/BIG-QUERY/cov_df.csv', index=False)

## Feature Engineering

### This is the place in this notebook where I call it a day on Sunday, 8/2/2020 as I move on to other models to complete.  I will revisit this notebook this evening and try to complete some EDA, Feature Engineering, and some Classification modeling

Before we create new columns to use to help train our model, let's take a look at the empty columns that need to be predicted in the end.

In [ ]:
test_df = df[['People_Hospitalized', 'Hospitalization_Rate', 'Confirmed', 'Deaths']][df.People_Hospitalized.isna()]
test_df

In [ ]:
test_df = df[df.People_Hospitalized.isna()]
test_df.columns

In [ ]:
# Create a train_df by dropping the test_df from df
train_df = df.drop(test_df.index)

In [ ]:
# Pull data from Wikipedia showing the population of each state or territory
# Create a Testing_Rate column that is the percentage of the population that has been tested
# Create a Positive_tet_Rate that is the number of positive tests / number of tests given

## Modeling

1. Split up the remaining 37 rows of data from our training dataset into a training and a validation set
2. Fit a linear regression model to the data and find the Root Mean Sqared Error of the model
3. Play with the features of the model to lower the RMSE
4. Run the model on our testing data to predict the hospitalization rate of people in our test states

In [ ]:
# import libraries
from sklearn.linear_model          import LinearRegression
from sklearn.metrics               import r2_score, mean_squared_error
from sklearn.model_selection       import train_test_split


In [ ]:
train_df.columns

In [ ]:
# Various combinations of features to be used in the model:
feat_0 = train_df.drop(columns=(['People_Hospitalized', 'Hospitalization_Rate'])).columns
feat_1 = train_df[['Confirmed', 'Deaths', 'Active', 'Incident_Rate', 'People_Tested', 'Testing_Rate']].columns
feat_2 = train_df[['Deaths', 'Active', 'Incident_Rate']].columns

# Toggle this to change our model predictions below
features = feat_1

In [ ]:
def model_results(feats, y=train_df['People_Hospitalized']):
    '''
    returns:      a Linear Regression model
    prints:       a printout of the RMSE
    feats         the features list to use on this particular model
    y:            a list of the targets for training and validation
    WARNING: This function as written only works in this notebook, must make adjustments 
    '''
    import pandas as pd
    import numpy as np
    
    from sklearn.linear_model          import LinearRegression
    from sklearn.metrics               import mean_squared_error
    from sklearn.model_selection       import train_test_split
    
    # Create the test-train split
    X = train_df[feats]
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    
    # Create and fit the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Find and printout RMSE
    val_predictions = model.predict(X_val)
    RMSE = (mean_squared_error(y_val, model.predict(X_val)))**(1/2)
    print(f"The Root Mean Squared Error for this model is {round(RMSE,2)}, meaning that it's accurate \
give or take up to {round(RMSE, 0)} people hospitalized in a given state, or so.")
    
    return model

In [ ]:
print("feature set 1:  ")
model_results(feat_0)
print("feature set 2:  ")
model_results(feat_1)
print("feature set 3:  ")
model_results(feat_2)

## Try out the new function on a new feature set

In [ ]:
second_model = model_results()

Now predict the missing states in the test dataframe:

In [ ]:
X_test = test_df[features]
test_df['People_Hospitalized'] = second_model.predict(X_test)
test_df[['Confirmed', 'Deaths', 'Active', 'Incident_Rate', 'People_Tested', 'People_Hospitalized']]

## Plot the results

In [ ]:
result_df = pd.concat([train_df, test_df])

In [ ]:
result_df.sort_values(by=['People_Hospitalized'], inplace=True)

In [ ]:
plt.figure(figsize=(21, 7))

plt.plot(result_df.index, result_df.People_Hospitalized);

plt.title('People Hospitalized by State')

In [ ]:
plt.figure(figsize=(21, 7))

sns.distplot(result_df['People_Hospitalized'], kde=False, bins = 55)

plt.title('Distribution of People Hospitalized by State');

In [ ]:
plt.figure(figsize=(21, 7))

sns.distplot(result_df['Deaths'], kde=False, bins = 55)

plt.title('Distribution of Deaths by State');

The coefficients above show us which features seem to indicate major or minor effects on the fit of the model

## Predict the Hospitalization numbers in the test states:

In [ ]:
X_test = test_df[features]

test_df['People_Hospitalized'] = second_model.predict(X_test)

In [ ]:
test_df

In [ ]:
test_df['Hospitalization_Rate'] = test_df['People_Hospitalized'] / test_df['Confirmed']

In [ ]:
test_df

## Issues that should be brought up:

The Mortality Rate and Hospitalization Rate in this data is inherenlty flawed as it is based upon the raw testing numbers in a given state. In order to do an apples to apples comparisson between the states, we need to really look at the hospitalization number divided by the population of the state.  The same should be done with the raw number of deaths. Simply basing any data on the the testing data can give a false sense of gloom or a false sense of security to any local population. We do have the testing rate, which is the number of people tested per 100,000 people, and we should use this number along with the state's population numbers to find our more tangible comparative numbers.

A better look at the data should be done by taking in to account the populations of each area and how the raw death and raw hospitalization numbers compare. From there we can do comparative analysis on population density, age, socio-econmic issues, and relative health prior to the pandemic. We have Incidents_Rate, but that only takes in to account the total positve tests / total population (it's actually cases per 100,000 people) and does not take in more solid numers such as hospitalizations and deaths.

## One more model:
Let's run the drill on one more model

In [ ]:
features = feat_1
experiment_model = model_results(feat_1)

In [ ]:
features = feat_2
third_model = model_results(feat_2)

The result of a more stripped down model was a RMSE that was 300 people higher than the second model. We should engineer some features in order to try and get a better model.

Also, given that we are only using state-level data instead of county or zip code level data, we are unlikely to be able to create a very good model.


In [ ]:
features = feat_0

In [ ]:
model = model_results(feat_0)